In [1]:
from poker import Range
from poker.hand import Combo
import holdem_calc

import holdem_functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
import dask
from dask import delayed
import multiprocessing as mp
import numpy as np

### Optimal bet using Kelly Criterion

In [2]:
def calc_kelly(win_odds, bet_odds):
    kelly_bet = round((bet_odds*win_odds-(1-win_odds))/bet_odds, 3)
    return kelly_bet

### Calculate the poker bet based on your hand, the board, and the villain range

In [99]:
def calc_poker_bet(my_hand, board, bankroll, return_multiplier, min_kelly = .5, recommended_kelly = .7, 
                   epsilon_hr = .15, max_kelly = 1.2, villain_range = Range('22+, KJ+ QJ+ JJ+ KT+ AT+, QT+ TJ+'), 
                   calc_villain_range = True, use_dask = True, villain_hand = None,
                   exact_calculation = True, verbose = True, num_sims = 1, read_from_file = None, 
                   print_elapsed_time = True):
    
    """
    Output
    First element of odds contains my probability of winning/tie/lose 
    Second element of odds contains my probablities
    Third element of odds contains villain probabilities
    """
    
    # @param my_hand is my hand in the format of Combo('AdKs') - must be 2 cards
    # @param board is a list of cards in the format ['Kd', 'Ah', '2c']
    
    # @param  bankroll is my current bankroll 
    # @param return_multiplier is the kelly return multiplier (e.g. if you win 2:1 then return_multiplier is 2)
    # @params - exact_calculation, num_sims, read_from_file, villain_hand, verbose, print_elapsed_time will stay default
    # @param villain_range is the range of possible hands that the villain has - must be in the format Range('77+, AT+, KJ+')
    
    odds = holdem_calc.calculate_odds_villan(board = board, exact = exact_calculation, 
                                             num = num_sims, input_file = read_from_file, 
                                             hero_cards = my_hand, villan_cards = villain_hand, 
                                             verbose = verbose, print_elapsed_time = print_elapsed_time)
    
    def calc_items(villain_combos=villain_range.combos):
    
            items = [holdem_calc.calculate_odds_villan(board = board, exact = exact_calculation, 
                                                       num = num_sims, input_file = read_from_file, 
                                                       hero_cards = my_hand, villan_cards = villain_hand, 
                                                       verbose = verbose, print_elapsed_time = print_elapsed_time) 
                     for villain_hand in villain_combos]
 
            return items

    if calc_villain_range == True:
        
        ######### Calc items #########
        
        if use_dask == False:
            
            items = calc_items()
            
        else:
            
            num_cores = mp.cpu_count()
            delayed_list = []
            start_pos = 0
            end_pos = int(np.floor(len(villain_range.combos) / num_cores))
            chunk_len = int(np.floor(len(villain_range.combos) / num_cores))

            for chunk in range(num_cores):
                if chunk != num_cores - 1:
                    delayed_list.append(delayed(calc_items)(villain_range.combos[start_pos:end_pos]))
                    start_pos = start_pos + chunk_len
                    end_pos = end_pos + chunk_len

                else:
                    delayed_list.append(delayed(calc_items)(villain_range.combos[start_pos:]))
            
            items = list(np.concatenate(dask.compute(*delayed_list)))
            
        
        ######### End Calc items #########
        
#         for hand_ranking in holdem_functions.hand_rankings:
#             print(hand_ranking +": " + str(np.mean([res[1][1][hand_ranking] for res in items if res])))
        
        hr = [hand_ranking +": " + str(np.mean([res[1][1][hand_ranking] for res in items if res])) 
              for hand_ranking in holdem_functions.hand_rankings]
    
        range_odds = {}
        
        [range_odds.update({odd_type: np.mean([res[0][odd_type] for res in items if res])}) for odd_type in ["tie", "win", "lose"]]
     
    
        ########################### START RANGE ###########################
        
        ######### Random Poker Hand Kelly Bet ############
        kelly_bet_random = calc_kelly(win_odds = odds[0]['win'], bet_odds = return_multiplier) # Assumes all players cards are random
        
        min_kelly_random = kelly_bet_random*min_kelly
        recommended_kelly_random = kelly_bet_random*recommended_kelly
        true_kelly_random = kelly_bet_random
        max_kelly_random = kelly_bet_random*max_kelly
        

        ######### Range-based Poker Hand Kelly Bet ############
        
        # kelly_bet_range assumes players cards are better than random - it tries to estimate the potential range 
        # of the villains cards since the villain did not fold (and likely continue to bet)
        kelly_bet_range = calc_kelly(win_odds = range_odds['win'], bet_odds = return_multiplier)
        min_kelly_range = kelly_bet_range*min_kelly
        recommended_kelly_range = kelly_bet_range*recommended_kelly
        true_kelly_range = kelly_bet_range
        max_kelly_range = kelly_bet_range*max_kelly
        
        ########################### END RANGE ###########################
        
        kelly_recommendations = {'min_kelly_random':min_kelly_random*bankroll,
                                 'recommended_kelly_random':recommended_kelly_random*bankroll,
                                 'true_kelly_random':true_kelly_random*bankroll, 
                                 'max_kelly_random':max_kelly_random*bankroll,
                                 'min_kelly_range':min_kelly_range*bankroll,
                                 'recommended_kelly_range':recommended_kelly_range*bankroll,
                                 'true_kelly_range':true_kelly_range*bankroll, 
                                 'max_kelly_range':max_kelly_range*bankroll}
        
        
        
#         return {'kelly_recommendations':kelly_recommendations, 'odds':odds, 'hand_ranking':hr, 'items':items} # items too big
        
        poker_stats = {'kelly_recommendations':kelly_recommendations, 'range_odds':range_odds, 'odds':odds, 'hand_ranking':hr}    
    
    
        ########################### START HAND RANKINGS ###########################

        hand_rankings = {
            'High Card': 0,
            'Pair': 1,
            'Two Pair': 2,
            'Three of a Kind': 3,
            'Straight': 4,
            'Flush': 5,
            'Full House': 6,
            'Four of a Kind': 7,
            'Straight Flush': 8,
            'Royal Flush': 9
        }
        
        my_likely_best_hand = max(poker_stats['odds'][1][0].items(), key=lambda k: k[1])
        villain_likely_best_hand = max(poker_stats['odds'][1][1].items(), key=lambda k: k[1])
        
        if hand_rankings[my_likely_best_hand[0]] > hand_rankings[villain_likely_best_hand[0]]: # if my is hand better than the villains hand
            if (my_likely_best_hand[1] - villain_likely_best_hand[1]) > (epsilon_hr): # if prob. of me winning is > than epsilon_hr
                final_note = '\n****** GOOD TO BET *****\n'
                print(final_note)
            else:
                final_note = '\n****** WARNING THE VILLAIN HAND WILL LIKELY WIN ******\n'
                print(final_note)
        else:
            final_note = '\n****** WARNING THE VILLAIN HAND WILL LIKELY WIN ******\n'
            print(final_note)
        
        
        display(HTML(villain_range.to_html()))
        
        
        ########################### END HAND RANKINGS ###########################
    
        poker_stats['Final Note'] = final_note
        return(poker_stats)
    
    else:
        
        print('\nWarning - not considering range underestimates the players cards. Consider setting calc_villain_range to True\n')
        items = calc_items()
        hr = [hand_ranking +": " + str(np.mean([res[1][1][hand_ranking] for res in items if res])) 
              for hand_ranking in holdem_functions.hand_rankings]
        kelly_bet = calc_kelly(win_odds = odds[0]['win'], bet_odds = return_multiplier)
        min_kelly = kelly_bet*min_kelly
        recommended_kelly = kelly_bet*recommended_kelly
        true_kelly = kelly_bet
        max_kelly = kelly_bet*max_kelly
        
        kelly_recommendations = {'min_kelly':min_kelly*bankroll,
                                 'recommended_kelly':recommended_kelly*bankroll,
                                 'true_kelly':true_kelly*bankroll,
                                 'max_kelly':max_kelly*bankroll}

        return {'kelly_recommendations':kelly_recommendations, 'odds':odds, 'hand_ranking':hr}

In [145]:
%%time
kelly_recommender = calc_poker_bet(my_hand=Combo('AhKs'), board = ['9s', '2d', 'As', 'Jh'], bankroll = 100, 
                                   use_dask = False, return_multiplier=2, calc_villain_range = True,
                                   villain_range = Range('22+, KJ+ QJ+ JJ+ KT+ AT+, QT+ TJ+'),
                                   print_elapsed_time=False)


****** WARNING THE VILLAIN HAND WILL LIKELY WIN ******



AA,AKs,AQs,AJs,ATs,,,,,,,,
AKo,KK,KQs,KJs,KTs,,,,,,,,
AQo,KQo,QQ,QJs,QTs,,,,,,,,
AJo,KJo,QJo,JJ,JTs,,,,,,,,
ATo,KTo,QTo,JTo,TT,,,,,,,,
,,,,,99,,,,,,,
,,,,,,88,,,,,,
,,,,,,,77,,,,,
,,,,,,,,66,,,,
,,,,,,,,,55,,,
,,,,,,,,,,44,,


Wall time: 650 ms


In [175]:
win_probability_output = pd.merge(
    pd.DataFrame(kelly_recommender['odds'][0].items(), columns = ['result', 'prob_winning']),
    pd.DataFrame(kelly_recommender['range_odds'].items(), columns = ['result', 'prob_winning_range'])
    )

kelly_bet_output = pd.DataFrame(kelly_recommender['kelly_recommendations'].items(), columns = ['kelly_type', 'kelly_bet'])

hand_probability_output = pd.merge(pd.DataFrame(kelly_recommender['odds'][1:][0][0].items(), columns=['hand', 'probability_my_hand']),
         pd.DataFrame(kelly_recommender['odds'][1:][0][1].items(), columns=['hand', 'probability_opponent_hand']),
        on = 'hand')

my_hand_value = hand_probability_output[hand_probability_output['probability_my_hand'] == hand_probability_output['probability_my_hand'].max()].index[0]
opponent_hand_value = hand_probability_output[hand_probability_output['probability_opponent_hand'] == hand_probability_output['probability_opponent_hand'].max()].index[0]

if my_hand_value > opponent_hand_value:
    print('\n ****** BY HAND PROBABILITIES - YOU HAVE A HIGHER PROBABILITY OF WINNING ESTIMATED {} HANDS BETTER THAN OPPONENT ({}-{}) ****** \n'.format(my_hand_value-opponent_hand_value, my_hand_value, opponent_hand_value))
elif my_hand_value == opponent_hand_value:
    if hand_probability_output.iloc[my_hand_value]['probability_my_hand'] > hand_probability_output.iloc[opponent_hand_value]['probability_opponent_hand']:
        print('\n ****** BY HAND PROBABILITIES - YOU HAVE A HIGHER PROBABILITY OF WINNING ESTIMATED {} HANDS BETTER THAN OPPONENT ({}-{}) ****** \n'.format(my_hand_value-opponent_hand_value, my_hand_value, opponent_hand_value))
    else:
        print('\n ****** BY HAND PROBABILITIES - YOU HAVE A LOWER PROBABILITY OF WINNING ESTIMATED {} HANDS BETTER THAN OPPONENT ({}-{}) ****** \n'.format(my_hand_value-opponent_hand_value, my_hand_value, opponent_hand_value))
else:
    print('\n ****** BY HAND PROBABILITIES - YOU HAVE A HIGHER PROBABILITY OF LOSING ESTIMATED {} HANDS WORSE THAN OPPONENT ({}-{}) ****** \n'.format(my_hand_value-opponent_hand_value, my_hand_value, opponent_hand_value))

display(HTML(win_probability_output.to_html()))
display(HTML(kelly_bet_output.to_html()))
display(HTML(hand_probability_output.to_html()))


 ****** BY HAND PROBABILITIES - YOU HAVE A HIGHER PROBABILITY OF WINNING ESTIMATED 0 HANDS BETTER THAN OPPONENT (1-1) ****** 



,result,prob_winning,prob_winning_range
0,tie,0.005797,0.037037
1,win,0.885639,0.794332
2,lose,0.108564,0.168631


,kelly_type,kelly_bet
0,min_kelly_random,41.40
1,recommended_kelly_random,57.96
2,true_kelly_random,82.80
3,max_kelly_random,99.36
4,min_kelly_range,34.55
5,recommended_kelly_range,48.37
6,true_kelly_range,69.10
7,max_kelly_range,82.92


,hand,probability_my_hand,probability_opponent_hand
0,High Card,0.000000,0.305336
1,Pair,0.695652,0.494862
2,Two Pair,0.260870,0.145520
3,Three of a Kind,0.043478,0.025231
4,Straight,0.000000,0.015613
5,Flush,0.000000,0.007905
6,Full House,0.000000,0.005336
7,Four of a Kind,0.000000,0.000198
8,Straight Flush,0.000000,0.000000
9,Royal Flush,0.000000,0.000000
